Stemming on Augmented Description and create a bigrams using gensim Phrase
- remove some plurials

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import unicodedata as uni
import re
def normalize(string):
    return uni.normalize('NFKD',string).encode('ascii','ignore')
from sys import stdout

In [2]:
TRAINING_SET_PATH = 'Data/cleantrainV2.csv'
TEST_SET_PATH = 'Data/cleantestV2.csv'
SEPARATOR = ','
INDEX_COL = 'Identifiant_Produit'
DESC_COL = 'Description'
LIBE_COL = 'Libelle'
MARQUE_COL = 'Marque'
MAX_PRICE = 1000
PRICE_COL = 'prix'
BIGRAM_COL = 'Categorie1'
AUGMENTED_DESC_COL = 'Description+Libelle'
SAVE_TRAINING_PATH = 'Data/cleantrainV3.csv'
SAVE_TEST_PATH = 'Data/cleantestV3.csv'
STOPLIST = set(('un une le les la au aux de des ce se ma mon ton ' +
               'sa the of then it ou and tres sur on en in dans ' +
               'pour for with avec et ne pas tout all without ' +
               'sans du il votre son one je vous nos ses vos est ' +
               'tu es cet cette tout toute comme comment sommes notre elle ils elles').split())

In [3]:
print "loading Data"
Ctrain = pd.read_csv(TRAINING_SET_PATH, sep=SEPARATOR,index_col=INDEX_COL)
Ctrain.fillna(' ',inplace=True)
Ctest = pd.read_csv(TEST_SET_PATH, sep=SEPARATOR,index_col=INDEX_COL)
Ctest.fillna(' ',inplace=True)
Ctrain.head(5)

loading Data


,Categorie1,Categorie2,Categorie3,Description,Libelle,Marque,Produit_Cdiscount,prix
Identifiant_Produit,,,,,,,,
0,1000014006,1000015308,1000015309,de collectif aux editions solesmes,benedictions de l eglise,aucunemarque,0,3.70
1,1000014006,1000015308,1000015309,de collectif aux editions solesmes,notice de st benoit lot de,aucunemarque,0,0.95
2,1000014006,1000015308,1000015309,de richard a m pe aux editions tequi,signes pour notre temps fatima,aucunemarque,0,21.85
3,1000014006,1000015308,1000015309,de mocquereau dom andr aux editions solesmes,nombre musical t,aucunemarque,0,26.60
4,1000014006,1000015308,1000015309,de collectif aux editions ns video,creation de l homme v,aucunemarque,0,7.12


In [4]:
def stemmer_line(line,stop):
    new_line =[]
    for word in line.split():
        if not word in stop and len(word)>1:
            length = len(word)
            stem_word = word
            if length > 2 and word[-1]=='s': 
                stem_word = stem_word[:-1]
                length -= 1
        new_line.append(word)
    return new_line

def stemmer(word):
    length = len(word)
    stem_word = word
    if length > 2 and word[-1]=='s': 
        stem_word = stem_word[:-1]
        length -= 1
    return stem_word

In [5]:
print "stem the words and remove stop words"
cuts = np.r_[np.arange(0,15000001,1000000),[15786885]]
print "on Description"
for i in range(cuts.shape[0]-1):
    Ctrain.ix[cuts[i]:cuts[i+1]-1,DESC_COL] = Ctrain.ix[cuts[i]:cuts[i+1]-1,DESC_COL].apply(lambda x: " ".join([stemmer(word) for word in x.split() if not word in STOPLIST and len(word)>1]))
    stdout.write("\r\x1b[K" + "[" + (i+1) * "--" + (16-(i+1))*"  " + "] %.2f" %((i+1)*6.25))
    stdout.flush()
Ctest[DESC_COL] = Ctest[DESC_COL].apply(lambda x: " ".join([stemmer(word) for word in x.split() if not word in STOPLIST and len(word)>1]))
print " "
print "on Libelle"
for i in range(cuts.shape[0]-1):
    Ctrain.ix[cuts[i]:cuts[i+1]-1,LIBE_COL] = Ctrain.ix[cuts[i]:cuts[i+1]-1,LIBE_COL].apply(lambda x: " ".join([stemmer(word) for word in x.split() if not word in STOPLIST and len(word)>1]))
    stdout.write("\r\x1b[K" + "[" + (i+1) * "--" + (16-(i+1))*"  " + "] %.2f" %((i+1)*6.25))
    stdout.flush()
Ctest[LIBE_COL] = Ctest[LIBE_COL].apply(lambda x: " ".join([stemmer(word) for word in x.split() if not word in STOPLIST and len(word)>1]))
print " "
print "Done"

stem the words and remove stop words
on Description
[--------------------------------] 100.00 
on Libelle
[--------------------------------] 100.00 
Done


In [ ]:
import gensim
bigramDesc = gensim.models.phrases.Phrases()
print "create bigrams vocabulary on Description"
for i in range(cuts.shape[0]-1):
    bigramDesc.add_vocab(Ctrain.ix[cuts[i]:cuts[i+1]-1,DESC_COL].apply(lambda x: x.split()))
    stdout.write("\r\x1b[K" + "[" + (i+1) * "--" + (16-(i+1))*"  " + "] %.2f" %((i+1)*6.25))
    stdout.flush()

print " "
bigramLibe = gensim.models.phrases.Phrases()
print "create bigrams vocabulary on Libelle"
for i in range(cuts.shape[0]-1):
    bigramLibe.add_vocab(Ctrain.ix[cuts[i]:cuts[i+1]-1,LIBE_COL].apply(lambda x: x.split()))
    stdout.write("\r\x1b[K" + "[" + (i+1) * "--" + (16-(i+1))*"  " + "] %.2f" %((i+1)*6.25))
    stdout.flush()

WARNING (theano.sandbox.cuda): CUDA is installed, but device gpu is not available  (error: Unable to get the number of gpus available: no CUDA-capable device is detected)


In [8]:
bigramLibe = gensim.models.phrases.Phrases()
print "create bigrams vocabulary on Libelle"
for i in range(cuts.shape[0]-1):
    bigramLibe.add_vocab(Ctrain.ix[cuts[i]:cuts[i+1]-1,LIBE_COL].apply(lambda x: x.split()))
    stdout.write("\r\x1b[K" + "[" + (i+1) * "--" + (16-(i+1))*"  " + "] %.2f" %((i+1)*6.25))
    stdout.flush()

create bigrams vocabulary on Description
[--------------------------------] 100.00

In [9]:
def detect_bigram(line,bigram):
    return " ".join(bigram[line.split()])

print "create bigrams on description"
for i in range(cuts.shape[0]-1):
    Ctrain.ix[cuts[i]:cuts[i+1]-1,DESC_COL] = Ctrain.ix[cuts[i]:cuts[i+1]-1,DESC_COL].apply(lambda x: detect_bigram(x,bigramDesc))
    stdout.write("\r\x1b[K" + "[" + (i+1) * "--" + (16-(i+1))*"  " + "] %.2f" %((i+1)*6.25))
    stdout.flush()
Ctest[DESC_COL] = Ctest[DESC_COL].apply(lambda x: detect_bigram(x,bigramDesc))
print " "

print "create bigrams on libelle"
for i in range(cuts.shape[0]-1):
    Ctrain.ix[cuts[i]:cuts[i+1]-1,LIBE_COL] = Ctrain.ix[cuts[i]:cuts[i+1]-1,LIBE_COL].apply(lambda x: detect_bigram(x,bigramLibe))
    stdout.write("\r\x1b[K" + "[" + (i+1) * "--" + (16-(i+1))*"  " + "] %.2f" %((i+1)*6.25))
    stdout.flush()
Ctest[LIBE_COL] = Ctest[LIBE_COL].apply(lambda x: detect_bigram(x,bigramLibe))

create bigrams on description
[--------------------------------] 100.00 
create bigrams on libelle
[--------------------------------] 100.00

In [10]:
Ctrain.to_csv(SAVE_TRAINING_PATH)
Ctest.to_csv(SAVE_TEST_PATH)

In [ ]:
Ctest

,Description,Libelle,Marque,prix
Identifiant_Produit,,,,
15786885,pasti_marseille vendu_unite quant,pasti nnmq cl,nnmq,16.99
15786886,baignoire droite nue surface acrylique renforc...,baignoire nue danubio surface,aucunemarque,149.00
15786887,baignoire droite nue surface acrylique renforc...,baignoire nue danubio surface,aucunemarque,163.99
15786888,portique hippo gym partir_moi encourager bebe ...,chiccomq portique hippo gym,chiccomq,349.00
15786889,lunette_soleil type wayfarer resine noire verr...,lunette_soleil mixte,aucunemarque,8.52
15786890,indice protection,lunette_soleil mixte,aucunemarque,9.31
15786891,lot pastille culture,lot pastille culture,aucunemarque,3.10
15786892,cartable_soufflet compatible pc colori rouge,cartable,delseymq,75.99
15786893,cartable_soufflet compatible pc colori chamoi,cartable,delseymq,85.99
